Create Nodes and Relationships:
Product Nodes: Each product has a node with properties like name, brand, etc.
Ingredient Nodes: Each ingredient has a node with properties that indicate whether it is vegan, halal, etc.
Contains Relationships: Create relationships from each Product node to its Ingredient nodes.

Label Dietary Restrictions:
Dietary Restriction Nodes: Create nodes for each dietary restriction (e.g., Vegan, Halal) for Yes and Maybe.
Suitable_For Relationships: create relationships from Products to these nodes based on the ingredient properties.

Allergens:
Allergen Nodes: Create nodes for each allergen (e.g. soy, gluten), for yes & maybe.
Contains_Allergen for relationship. Relationship property contains notes on freq.

Source:
Source Node: Showcase source Plant, Animal, Mineral, Synthetic. 

In [6]:
#libraries
import sys
from neo4j import GraphDatabase
import pandas as pd
import logging
from datetime import datetime

In [7]:
if len(sys.argv) > 1:
    QueryFolder = sys.argv[1]

# Connection to local database
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "1nonly1uNeed")



with GraphDatabase.driver(uri=URI, auth=AUTH) as driver:
    driver.verify_connectivity()


In [8]:
# to reset the db entirely for a fresh start.
# edit the match at the end to include the nodes to remove/ not remove. 
# deletes associated relationships.
class Neo4jConnection:

    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__password = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__password))
            self.__driver.verify_connectivity()  # Verifying connectivity
            print("Connected Successfully")
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def reset_database(self):
        with self.__driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
            print("Database reset successfully.")

    def query(self, cypher_query, parameters=None):
        assert self.__driver is not None, "Driver not initialized!"
        with self.__driver.session() as session:
            return session.run(cypher_query, parameters)

# delete specific nodes, change within the MATCH  
with driver.session() as session:
    session.run("MATCH (i:Ingredient) DETACH DELETE i")
    session.run("MATCH (i:Brand) DETACH DELETE i")
    session.run("MATCH (i:Product) DETACH DELETE i")
    session.run("MATCH (i:DietaryRestriction) DETACH DELETE i")
    session.run("MATCH (i:AlcoholStatus) DETACH DELETE i")
    session.run("MATCH (i:ProcessingType) DETACH DELETE i")
    session.run("MATCH (i:Allergen) DETACH DELETE i")
    session.run("MATCH (i:Category) DETACH DELETE i")
    session.run("MATCH (i:Intolerance) DETACH DELETE i")
    session.run("MATCH (i:ManufacturingStep) DETACH DELETE i")
    session.run("MATCH (i:Other) DETACH DELETE i")
    

C:\Users\apple\AppData\Local\Temp\ipykernel_10124\1438698596.py:33: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [9]:
# setting up dataframes. 

# P&S 100 product list
dfp = pd.read_csv('C:/Users/apple/Desktop/Neo4j/P&S_Pantry_100_items_v4.csv')

# Ingredient db
dfi = pd.read_csv('C:/Users/apple/Desktop/Neo4j/all_unique_ingredients_properties_v2.csv')

#Allergen db
dfa =pd.read_csv('C:/Users/apple/Desktop/Neo4j/Allergens_list.csv')

# Ingredients are now products & their ingredients (l2)
dfs =pd.read_csv('C:/Users/apple/Desktop/Neo4j/Ingredients of products DB.csv')

# Primary ingredients 
dft =pd.read_csv('C:/Users/apple/Desktop/Neo4j/unique_ingredients_v2.csv')

# Final Products manufacturing steps
dfe =pd.read_csv('C:/Users/apple/Desktop/Neo4j/Product_Manufacturing_Steps.csv')

In [10]:
#Create the unique brands   
unique_brands = dfp['Brand'].unique()

# Connect to Neo4j
with GraphDatabase.driver(uri=URI, auth=AUTH) as driver:
    with driver.session() as session:

        # Verify connectivity
        session.run("RETURN 1")
        print("Connected Successfully")

        # Create unique Brand nodes
        for brand in unique_brands:
            session.run("MERGE (b:Brand {name: $brand})", parameters={'brand': brand})

Connected Successfully


In [11]:
# Create the products, the quantity and ingredients are the properties. 
# The products have a relationship with the brand "part of". 
# The relationship starts at Product and ends with Brand.

# Function to create product nodes and relationships
def create_product(tx, name, quantity, ingredients, brand):
    tx.run("MERGE (p:Product {name: $name, quantity: $quantity, ingredients: $ingredients}) "
           "MERGE (b:Brand {name: $brand}) "
           "MERGE (p)-[:PART_OF]->(b)",
           name=name, quantity=quantity, ingredients=ingredients, brand=brand)

# Connect to Neo4j and create products
with GraphDatabase.driver(uri=URI, auth=AUTH) as driver:
    with driver.session() as session:
        # Loop through DataFrame and create products
        for index, row in dfp.iterrows():
            session.write_transaction(create_product, row['Name'], row['Quantity'], row['Ingredients'], row['Brand'])

C:\Users\apple\AppData\Local\Temp\ipykernel_10124\24983060.py:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_product, row['Name'], row['Quantity'], row['Ingredients'], row['Brand'])


In [12]:
# Create ingredients from dfi
def add_ingredient(tx, ingredient_name):
    tx.run("CREATE (:Ingredient {name: $name})", name=ingredient_name)

with driver.session() as session:
    for ingredient_name in dfi['U-Ingredients'].unique():  # Ensure unique ingredient names
        session.write_transaction(add_ingredient, ingredient_name)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\291556516.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\291556516.py:7: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_ingredient, ingredient_name)


In [13]:
# Take an existing ingredient node and updates it with additional properties, found in columns of dfi.

def add_ingredient_properties(tx, ingredient_name, alt_name, related_ingredients, description, manufacturing):
    tx.run("""
    MATCH (i:Ingredient {name: $name})
    SET i.alternateName = $alt_name, 
        i.relatedIngredients = $related_ingredients, 
        i.description = $description, 
        i.manufacturing = $manufacturing
    """, 
    name=ingredient_name, 
    alt_name=alt_name, 
    related_ingredients=related_ingredients, 
    description=description, 
    manufacturing=manufacturing)

with driver.session() as session:
    for index, row in dfi.iterrows():
        ingredient_name = row['U-Ingredients']
        alt_name = row['Alternate Name'] if pd.notna(row['Alternate Name']) else None
        related_ingredients = row['Related Ingredients'] if pd.notna(row['Related Ingredients']) else None
        description = row['Description'] if pd.notna(row['Description']) else None
        manufacturing = row['Manufacturing'] if pd.notna(row['Manufacturing']) else None

        session.write_transaction(add_ingredient_properties, ingredient_name, alt_name, related_ingredients, description, manufacturing)



C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2380170666.py:17: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2380170666.py:25: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_ingredient_properties, ingredient_name, alt_name, related_ingredients, description, manufacturing)


In [14]:
#create relationship between products and ingredients called "made of"
def create_relationship(tx, product_name, ingredient_name):
    query = (
        "MATCH (p:Product {name: $product_name}), (i:Ingredient {name: $ingredient_name}) "
        "MERGE (p)-[:MADE_OF]->(i)"
    )
    tx.run(query, product_name=product_name, ingredient_name=ingredient_name)

with driver.session() as session:
    # Iterate over each row in the DataFrame
    for index, row in dfp.iterrows():
        product_name = row["Name"]  
        for i in range(1, 151):  
            ingredient_col = f"Ingredient - {i}"
            if pd.notna(row[ingredient_col]):  # Check if the ingredient cell is not empty
                session.write_transaction(create_relationship, product_name, row[ingredient_col])

driver.close()

C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2836830410.py:9: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2836830410.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, product_name, row[ingredient_col])


In [15]:
# Dietary restriction categories with specific names
nodes = [
    "Is_Halal", "Maybe_Halal",
    "Is_Vegan", "Maybe_Vegan",
    "Is_Vegetarian", "Maybe_Vegetarian",
    "Is_Kosher", "Maybe_Kosher"
]

# Create dietary restriction nodes with specific names
with driver.session() as session:
    for node_name in nodes:
        session.run("CREATE (:DietaryRestriction {name: $name})", name=node_name)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\133347853.py:10: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [16]:
# for the "Maybe", relationship called "Maybe_Suitable_For" with the "maybe" node for each dietary restriction. 
# the ingredient property for the dietary restriction is either "Yes" or "Maybe" for all the ingredients & contains atleast 1 "Maybe" and no "No"
# Determines the dietary status ('Is', 'Maybe', or 'No') of each product for each restriction based on its ingredients.

def determine_dietary_status(ingredient_list, dfi, restriction):
    has_yes = False
    has_maybe = False
    
    for ingredient in ingredient_list:
        ingredient_row = dfi[dfi['U-Ingredients'].str.contains(ingredient, na=False, regex=False)]
        if not ingredient_row.empty:
            status = ingredient_row[restriction].values[0]
            if status == 'No':
                return 'No'
            elif status == 'Yes':
                has_yes = True
            elif status == 'Maybe':
                has_maybe = True
    
    # Determine the status based on the ingredients
    if has_yes and not has_maybe:
        return 'Is'
    elif has_yes or has_maybe:  # It has a mix of 'Yes' and 'Maybe' or only 'Maybe'
        return 'Maybe'
    else:
        return 'No'  # No ingredients or none are marked as 'Yes' or 'Maybe'



In [17]:
# Create relationships ('Is_Suitable_For' or 'Maybe_Suitable_For') from products to the dietary restriction nodes based on the status.
# Add the Notes e.g. "Halal Notes" for "Halal" as a property of the relationship.
# Dietary restrictions to check
def create_dietary_relationship(tx, product_name, restriction, status):
    node_name = f"{status}_{restriction}"
    relationship_type = f"{status}_Suitable_For"
    tx.run("""
    MATCH (p:Product {name: $product_name})
    MATCH (d:DietaryRestriction {name: $node_name})
    MERGE (p)-[:""" + relationship_type + """]->(d)
    """, product_name=product_name, node_name=node_name)

# Dietary restrictions to check
dietary_restrictions = ['Halal', 'Vegan', 'Vegetarian', 'Kosher']

# Iterate over products and create relationships based on the determined status
with driver.session() as session:
    for index, product_row in dfp.iterrows():
        product_name = product_row['Name']
        ingredients = [ing.strip() for ing in product_row['Ingredients'].split(',')]
        
        for restriction in dietary_restrictions:
            status = determine_dietary_status(ingredients, dfi, restriction)
            if status != 'No':  # Proceed if status is 'Is' or 'Maybe'
                session.write_transaction(create_dietary_relationship, product_name, restriction, status)

C:\Users\apple\AppData\Local\Temp\ipykernel_10124\1290711878.py:17: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\1290711878.py:25: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_dietary_relationship, product_name, restriction, status)


In [18]:
# add properties to the existing relationships between ingredients and dietary restrictions. 
# These properties will be based on the notes associated with each dietary restriction.

# Function to add a property to a relationship based on the notes from the dfi DataFrame
def add_notes_to_relationship(tx, ingredient_name, restriction, status, note):
    # Only proceed if there is a note to add, and the status is 'Yes' or 'Maybe'
    if note and status in ['Yes', 'Maybe']:
        # Determine the relationship type based on the status
        relationship_type = "Is_Suitable_For" if status == 'Yes' else "Maybe_Suitable_For"
        
        # Create a Cypher query to add the note to the relationship
        query = f"""
        MATCH (i:Ingredient {{name: $ingredient_name}})
        MATCH (i)-[r:{relationship_type}]->(d:DietaryRestriction {{name: $restriction}})
        SET r.notes = $note
        """
        tx.run(query, ingredient_name=ingredient_name, restriction=restriction, note=note)

# Assume we have a list of ingredients and their dietary statuses
with driver.session() as session:
    for index, row in dfi.iterrows():
        ingredient_name = row['U-Ingredients']
        # Iterate through each dietary restriction to add notes
        for restriction in dietary_restrictions:
            # Build the name of the restriction and note columns
            restriction_column = restriction
            note_column = restriction + ' Notes'
            
            # Skip if the column is empty or the status is not 'Yes' or 'Maybe'
            if pd.isna(row[note_column]) or row[restriction_column] not in ['Yes', 'Maybe']:
                continue
        # Create the full name of the restriction node based on the status
        restriction_node_name = ("Is_" if row[restriction_column] == 'Yes' else "Maybe_") + restriction
        
        # Add the note to the relationship
        session.write_transaction(add_notes_to_relationship, ingredient_name, restriction_node_name, row[restriction_column], row[note_column])



C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2431955814.py:20: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2431955814.py:36: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_notes_to_relationship, ingredient_name, restriction_node_name, row[restriction_column], row[note_column])


In [19]:
# Create Nodes for Alcohol Status

alcohol_statuses = ["Contains_Alcohol", "Maybe_Contains_Alcohol"]

# Create alcohol status nodes
with driver.session() as session:
    for status in alcohol_statuses:
        session.run("MERGE (:AlcoholStatus {name: $name})", name=status)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\1634919363.py:6: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [20]:
# Link Products to the Appropriate Alcohol Status

# Function to determine if a product contains or maybe contains alcohol
def determine_alcohol_status(ingredient_list, dfi):
    has_yes = False
    has_maybe = False
    
    for ingredient in ingredient_list:
        ingredient_row = dfi[dfi['U-Ingredients'].str.contains(ingredient, na=False, regex=False)]
        if not ingredient_row.empty:
            alcohol_status = ingredient_row['Alcohol'].values[0]
            if alcohol_status == 'Yes':
                has_yes = True
            elif alcohol_status == 'Maybe':
                has_maybe = True
    
    if has_yes:
        return 'Contains_Alcohol'
    elif has_maybe:
        return 'Maybe_Contains_Alcohol'
    else:
        return None

# Function to create a relationship to the appropriate alcohol status node
def create_alcohol_relationship(tx, product_name, alcohol_status):
    tx.run("""
    MATCH (p:Product {name: $product_name})
    MATCH (a:AlcoholStatus {name: $alcohol_status})
    MERGE (p)-[:Contains_Or_Maybe_Contains_Alcohol]->(a)
    """, product_name=product_name, alcohol_status=alcohol_status)

# Iterate over products and create relationships based on alcohol content
with driver.session() as session:
    for index, product_row in dfp.iterrows():
        product_name = product_row['Name']
        ingredients = [ing.strip() for ing in product_row['Ingredients'].split(',')]
        
        alcohol_status = determine_alcohol_status(ingredients, dfi)
        if alcohol_status:
            session.write_transaction(create_alcohol_relationship, product_name, alcohol_status)

C:\Users\apple\AppData\Local\Temp\ipykernel_10124\1023264413.py:33: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\1023264413.py:40: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_alcohol_relationship, product_name, alcohol_status)


In [21]:
# Processing types nodes
processing_types = ["Processed", "Ultra-Processed", "Raw"]

# Create processing type nodes
with driver.session() as session:
    for processing_type in processing_types:
        session.run("MERGE (:ProcessingType {name: $name})", name=processing_type)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3885222572.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [22]:
# I want to know if the product's processing level based on the ingredients Processing Level. I think the best way to do this would be to:
#  •	Product is Raw because all the ingredients Processing “Raw”
#  •	Product is Processed because all the ingredients Processing is either “Raw”
#  •	Product is Ultra-Processed because it contains atleast one “Ultra-Processed” ingredient, the remainder ingredients can be a combination of “Raw” and “Processed” 

# Function to Determine the Product's Processing Level
def determine_product_processing_level(ingredients, dfi):
    has_raw = False
    has_processed = False
    has_ultra_processed = False
    
    for ingredient in ingredients:
        ingredient_info = dfi[dfi['U-Ingredients'].str.contains(ingredient, na=False, regex=False)]
        if not ingredient_info.empty:
            processing_type = ingredient_info.iloc[0]['Processing']
            if processing_type == 'Raw':
                has_raw = True
            elif processing_type == 'Processed':
                has_processed = True
            elif processing_type == 'Ultra-Processed':
                has_ultra_processed = True
    
    # Determine the product's processing level
    if has_ultra_processed:
        return 'Ultra-Processed'
    elif has_processed or (has_processed and has_raw):  # Any processed ingredient makes the product processed
        return 'Processed'
    elif has_raw and not (has_processed or has_ultra_processed):  # All ingredients are raw
        return 'Raw'
    else:
        return None  # Default case if none match or no ingredients


In [23]:
# Function to create a relationship to the appropriate processing type node
def create_processing_relationship(tx, product_name, processing_level):
    tx.run("""
    MATCH (p:Product {name: $product_name})
    MERGE (pt:ProcessingType {name: $processing_level})
    MERGE (p)-[:HAS_PROCESSING_LEVEL]->(pt)
    """, parameters={'product_name': product_name, 'processing_level': processing_level})




In [24]:
# Iterate Over Products and Create Relationships
with driver.session() as session:
    for index, product_row in dfp.iterrows():
        product_name = product_row['Name']
        # Assume 'Ingredients' column is a string of ingredient names separated by commas
        ingredients = [ing.strip() for ing in product_row['Ingredients'].split(',')]
        
        processing_level = determine_product_processing_level(ingredients, dfi)
        if processing_level:  # Ensure processing_level is not None
            session.write_transaction(create_processing_relationship, product_name, processing_level)



C:\Users\apple\AppData\Local\Temp\ipykernel_10124\414505372.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\414505372.py:10: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_processing_relationship, product_name, processing_level)


Allergens should be considered a seperate node type.
This allows the node to contain properties related to the allergen e.g. severity, incidence.
The data model becomes clearer.
I am seeing the need because manufacturing process can have cross contamination and then I can link this allergen node to it.

This will increase the complexity of the query and may also slow down the query.
There is then a risk of a double up, where an ingredient with the same name as the allergen exists. 
This can be refactored down the line.

In [25]:
# Allergens for each ingredient
# Each Ingredient specific allergen is listed under the columns "Allergen - 1" and "Allergen - 2"
# create Allergen nodes based on "Allergen - 1" and "Allergen - 2" columns the df "dfi"

def create_or_update_ingredient_and_allergen_nodes(tx, dfi):
    for index, row in dfi.iterrows():
        # Create or update Ingredient node
        tx.run("""
            MERGE (i:Ingredient {name: $ingredient_name})
            ON CREATE SET i.description = $description
            """, ingredient_name=row['U-Ingredients'], description=row['Description'])

        # Create Allergen nodes from Allergen - 1 and Allergen - 2
        for allergen_column in ['Allergen - 1', 'Allergen - 2']:
            allergen_name = row[allergen_column]
            if allergen_name and allergen_name != '-':
                tx.run("""
                    MERGE (a:Allergen {name: $allergen_name})
                    """, allergen_name=allergen_name)
                # Create relationship between Ingredient and Allergen
                tx.run("""
                    MATCH (i:Ingredient {name: $ingredient_name}), (a:Allergen {name: $allergen_name})
                    MERGE (i)-[:CONTAINS_ALLERGEN]->(a)
                    """, ingredient_name=row['U-Ingredients'], allergen_name=allergen_name)

with driver.session() as session:
    session.write_transaction(create_or_update_ingredient_and_allergen_nodes, dfi)




C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2751269177.py:26: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2751269177.py:27: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_or_update_ingredient_and_allergen_nodes, dfi)


In [26]:
# Relationship Creation and Updating Functions
# Create or Update Relationships Between Ingredients and Allergens: Based on the CONTAINS_ALLERGEN relationship from the dfi data.
# Update Product Nodes Based on Ingredient Allergens: For each Product node in dfp, update its allergen-related information based on the allergens present in its ingredients.

def create_ingredient_allergen_relationships(tx, dfi):
    for index, row in dfi.iterrows():
        # Allergen information is in 'Allergen - 1' and 'Allergen - 2'
        for allergen_column in ['Allergen - 1', 'Allergen - 2']:
            allergen_name = row[allergen_column]
            if allergen_name and allergen_name != '-':
                # Create relationship
                tx.run("""
                    MATCH (i:Ingredient {name: $ingredient_name})
                    MATCH (a:Allergen {name: $allergen_name})
                    MERGE (i)-[:CONTAINS_ALLERGEN]->(a)
                    """, ingredient_name=row['U-Ingredients'], allergen_name=allergen_name)
                
def update_product_with_ingredients(tx, dfp):
    for index, row in dfp.iterrows():
        product_name = row['Name']  # Assuming the product name is in the 'Name' column
        
        # Create or update the Product node
        tx.run("""
            MERGE (p:Product {name: $product_name})
            ON CREATE SET p.brand = $brand, p.quantity = $quantity
            ON MATCH SET p.brand = $brand, p.quantity = $quantity
            """, product_name=product_name, brand=row['Brand'], quantity=row['Quantity'])

        # Iterate over each ingredient column to associate the product with its ingredients
        for i in range(1, 145):  # 145 ingredient columns considered from dfp
            ingredient_name = row.get(f'Ingredient - {i}')
            if ingredient_name and ingredient_name.strip():
                # Create or update the Ingredient node
                tx.run("""
                    MERGE (ing:Ingredient {name: $ingredient_name})
                    """, ingredient_name=ingredient_name.strip())
                
                # Create or update the relationship between Product and Ingredient
                tx.run("""
                    MATCH (p:Product {name: $product_name})
                    MATCH (ing:Ingredient {name: $ingredient_name})
                    MERGE (p)-[:MADE_OF]->(ing)
                    """, product_name=product_name, ingredient_name=ingredient_name.strip())


# write to Neo4j
with driver.session() as session:
    session.write_transaction(create_ingredient_allergen_relationships, dfi)
    session.write_transaction(update_product_with_ingredients, dfp)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3500170047.py:47: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3500170047.py:48: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_ingredient_allergen_relationships, dfi)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3500170047.py:49: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(update_product_with_ingredients, dfp)


#Cypher queries to test allergens. 
# check if the Ingredient nodes and Allergen nodes are created and properly related
MATCH (i:Ingredient)-[:CONTAINS_ALLERGEN]->(a:Allergen)
RETURN i.name, a.name LIMIT 10

# check if the Product nodes are related to Ingredient nodes
MATCH (p:Product)-[:MADE_OF]->(i:Ingredient)
RETURN p.name, collect(i.name) LIMIT 10

# find products that contain a specific allergen
MATCH (p:Product)-[:MADE_OF]->(:Ingredient)-[:CONTAINS_ALLERGEN]->(a:Allergen {name: 'Peanut'})
RETURN p.name

# list all allergens for a specific product
MATCH (p:Product {name: 'Some Product'})-[:MADE_OF]->(i:Ingredient)-[:CONTAINS_ALLERGEN]->(a:Allergen)
RETURN a.name


In [27]:
#Create Indexes to speed up queries for product, ingredient and allergen for now.

# Define a function to create an index if it does not exist
def create_index(tx, label, property):
    query = (
        f"CREATE INDEX index_for_{label.lower()}_{property} IF NOT EXISTS "
        f"FOR (n:{label}) "
        f"ON (n.{property})"
    )
    tx.run(query)

# Define a function that calls the create_index function for different labels and properties
def setup_indexes(driver):
    with driver.session() as session:
        # Create an index for the Product nodes
        session.write_transaction(create_index, "Product", "name")
        
        # Create an index for the Ingredient nodes
        session.write_transaction(create_index, "Ingredient", "name")
        
        # Create an index for the Allergen nodes
        session.write_transaction(create_index, "Allergen", "name")

# Assuming driver is already defined and connected to Neo4j
setup_indexes(driver)

C:\Users\apple\AppData\Local\Temp\ipykernel_10124\4098167450.py:14: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\4098167450.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_index, "Product", "name")


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\4098167450.py:19: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_index, "Ingredient", "name")
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\4098167450.py:22: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_index, "Allergen", "name")


#view indexes using cypher
SHOW INDEXES

# Working on Intolerances below
applying probability to relationship. iteration 1  

In [28]:
# Given data as placeholder for probability of insensitivity.
data = {
    "Intolerance": ["Fructose", "Gluten", "Caffeine", "Lactose", "Sulfites"],
    "Rarity": ["Rare", "Common", "Rare", "Common", "Extremely Rare"],
    "Probability": [0.1, 0.5, 0.1, 0.5, 0.01]
}

# Create the DataFrame
df_intolerances = pd.DataFrame(data)

df_intolerances

,Intolerance,Rarity,Probability
0,Fructose,Rare,0.10
1,Gluten,Common,0.50
2,Caffeine,Rare,0.10
3,Lactose,Common,0.50
4,Sulfites,Extremely Rare,0.01


In [29]:
#create the Intolerance nodes
#Updates an existing or new Intolerance node with the rarity and probability properties

def create_intolerance_nodes(tx, df_intolerances):
    for index, row in df_intolerances.iterrows():
        tx.run("""
            MERGE (intol:Intolerance {name: $name})
            ON CREATE SET intol.rarity = $rarity, intol.probability = $probability
            """, 
            name=row['Intolerance'], rarity=row['Rarity'], probability=row['Probability'])

with driver.session() as session:
    session.write_transaction(create_intolerance_nodes, df_intolerances)

C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2838313863.py:12: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2838313863.py:13: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_intolerance_nodes, df_intolerances)


In [30]:
#create Sensitivity caused by relationship
#function will associate each Ingredient with its respective Intolerance nodes.
#set the probability property for the relationship.

def create_sensitivity_relationships(tx, dfi, df_intolerances):
    # Convert df_intolerances to a dictionary for quick lookups
    intolerance_probability = df_intolerances.set_index('Intolerance')['Probability'].to_dict()
    
    for index, row in dfi.iterrows():
        ingredient_name = row['U-Ingredients']
        
        # Check both intolerance columns
        for col in ['Intolerance - 1', 'Intolerance - 2']:
            intolerance_name = row.get(col)
            if pd.notna(intolerance_name) and intolerance_name in intolerance_probability:
                # Get the probability for this intolerance
                probability = intolerance_probability[intolerance_name]
                
                # Create the Sensitivity caused by relationship
                tx.run("""
                    MATCH (i:Ingredient {name: $ingredient_name})
                    MATCH (intol:Intolerance {name: $intolerance_name})
                    MERGE (i)-[r:SENSITIVITY_CAUSED_BY]->(intol)
                    SET r.probability = $probability
                    """, ingredient_name=ingredient_name, intolerance_name=intolerance_name, probability=probability)

with driver.session() as session:
    session.write_transaction(create_sensitivity_relationships, dfi, df_intolerances)

C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3110226331.py:27: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3110226331.py:28: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_sensitivity_relationships, dfi, df_intolerances)


In [31]:
#apparently i can find the list using python instead of cypher to verify its all worked.

def find_products_with_intolerance(tx, intolerance_name):
    query = """
    MATCH (p:Product)-[:MADE_OF]->(i:Ingredient)-[:SENSITIVITY_CAUSED_BY]->(intol:Intolerance {name: $intolerance_name})
    RETURN p.name AS ProductName, collect(i.name) AS IngredientsCausingSensitivity
    """
    results = tx.run(query, intolerance_name=intolerance_name)
    return [(record["ProductName"], record["IngredientsCausingSensitivity"]) for record in results]


In [32]:
# Replace 'Gluten' with the intolerance you're interested in
intolerance_of_interest = 'Caffeine'

with driver.session() as session:
    products_with_intolerance = session.read_transaction(find_products_with_intolerance, intolerance_of_interest)

for product, ingredients in products_with_intolerance:
    print(f"Product: {product} may cause sensitivity due to: {ingredients}")


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2939572601.py:4: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2939572601.py:5: DeprecationWarning: read_transaction has been renamed to execute_read
  products_with_intolerance = session.read_transaction(find_products_with_intolerance, intolerance_of_interest)


Product: Original Instant Coffee may cause sensitivity due to: ['Coffee Beans']
Product: Classic Medium Roast Instant Freeze Dried Coffee Refill may cause sensitivity due to: ['Coffee Beans']
Product: Instant Coffee Powder may cause sensitivity due to: ['Instant Coffee.']
Product: Blend Instant Coffee may cause sensitivity due to: ['Instant Coffee.']
Product: Blend Granulated Instant Coffee may cause sensitivity due to: ['Instant Coffee.']


#cypher queries to test the intolerance implementation

# Retrieve all products that contain ingredients with gluten intolerance.

MATCH (p:Product)-[:MADE_OF]->(i:Ingredient)-[:SENSITIVITY_CAUSED_BY]->(intol:Intolerance {name: 'Gluten'})
RETURN p.name AS ProductName, collect(i.name) AS IngredientsWithGluten

Next working on ingredient categorisation.

In [33]:


# Function to create Category nodes from dfi
def create_category_nodes(tx, dfi):
    # Extract unique categories from both 'Category 1' and 'Category 2'
    categories = set(dfi['Category 1'].dropna().unique().tolist() + dfi['Category 2'].dropna().unique().tolist())
    
    # Create nodes for each unique category
    for category in categories:
        if category:  # Ensure the category is not None or empty
            tx.run("MERGE (c:Category {name: $category})", category=category)



# Function to execute the transaction
def add_categories_to_db(dfi):
    with driver.session() as session:
        session.write_transaction(create_category_nodes, dfi)

# Call the function with the dfi DataFrame
add_categories_to_db(dfi)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\1161548726.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\1161548726.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_category_nodes, dfi)


In [34]:
#Create Ingredient-Category Relationships
def create_ingredient_category_relationships(tx, dfi):
    for index, row in dfi.iterrows():
        ingredient_name = row['U-Ingredients']
        categories = [row['Category 1'], row['Category 2']]  # List of potential categories
        
        for category in categories:
            if pd.notna(category):  # Check if the category is not NaN
                tx.run("""
                    MATCH (i:Ingredient {name: $ingredient_name})
                    MATCH (c:Category {name: $category})
                    MERGE (i)-[:BELONGS_TO]->(c)
                    """, ingredient_name=ingredient_name, category=category)

with driver.session() as session:
    session.write_transaction(create_ingredient_category_relationships, dfi)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\943830235.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\943830235.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_ingredient_category_relationships, dfi)


#Query Product Nodes Indirectly
##### write a query that allows us to retrieve all products along with their associated categories through the ingredients.
MATCH (p:Product)-[:MADE_OF]->(i:Ingredient)-[:BELONGS_TO]->(c:Category)
RETURN p.name AS ProductName, collect(DISTINCT c.name) AS Categories

#### Find Products Excluding a Specific Category
MATCH (p:Product)
WHERE NOT EXISTS {
    (p)-[:MADE_OF]->(:Ingredient)-[:BELONGS_TO]->(:Category {name: 'Colorant'})
}
RETURN p.name AS ProductName

#### List Ingredients of a Product by Category
MATCH (p:Product {name: 'ProductName'})-[:MADE_OF]->(i:Ingredient)-[:BELONGS_TO]->(c:Category)
RETURN i.name AS IngredientName, c.name AS CategoryName

#### Identify Products with All Natural Ingredients:
MATCH (p:Product)
WHERE ALL(i IN [(p)-[:MADE_OF]->(ing:Ingredient) | ing] WHERE NOT (ing)-[:BELONGS_TO]->(:Category {name: 'Artificial'}))
RETURN p.name AS ProductName


In [35]:
# Data Cleaning 'dfs' DataFrame
# Consolidate Ingredients
# - Merge ingredients from 'Ingredient - 1' to 'Ingredient - 6' into one list.

# Remove Duplicates
# - Use a set or similar method to ensure unique ingredients.

# Clean Data
# - Trim whitespace and standardize case (e.g., title case) for ingredient names.




In [36]:
# Define the columns that contain the ingredient information
ingredient_columns = [
    'Ingredient - 1', 'Ingredient - 2', 'Ingredient - 3',
    'Ingredient - 4', 'Ingredient - 5', 'Ingredient - 6'
]

# Extract all ingredients from the defined columns into a single list
all_ingredients = []
for col in ingredient_columns:
    # Append the non-null values from each column to the all_ingredients list
    all_ingredients.extend(dfs[col].dropna().unique().tolist())

# Remove duplicates by converting the list to a set and then back to a list
unique_ingredients = list(set(all_ingredients))


In [37]:
print(unique_ingredients[:10])  # Print the first 10 ingredients
unique_ingredients = list(set(all_ingredients))

# Create a DataFrame from the list
dfU = pd.DataFrame(unique_ingredients, columns=['Ingredient'])

# Specify the filename for your CSV
filename = "unique_ingredients.csv"

# Write the DataFrame to a CSV file
dfU.to_csv(filename, index=False)

print(f"File '{filename}' has been created with the list of unique ingredients.")

['Sugars (glucose, sucrose)', 'Corn', 'Caramelized sugar', 'Lemon', 'Potassium Hydroxide', 'Chicken bones', 'Beets', 'cheese ingredients', 'Sodium Carbonate', 'garlic oil']
File 'unique_ingredients.csv' has been created with the list of unique ingredients.


In [38]:

# Step 1: Process Unique Second-Tier Ingredients
# - In Neo4j, use MERGE to create nodes for these ingredients, avoiding duplicates.

# Step 2: Process Manufacturing Steps
# - Extract steps and descriptions from 'Manufacturing Step 1' to 'Manufacturing Step 5' and related columns.
# - Create unique nodes for each step with descriptions as node properties.

# Step 3: Create Relationships Between Ingredients and Manufacturing Steps
# - For each 'dfs' entry, link new ingredient nodes to corresponding manufacturing steps.
# - After the final step, connect to the Product node, matching the 'Product' column name with existing node names.


In [39]:
# Step 1
def add_ingredient(tx, ingredient_name):
    # Use MERGE to avoid creating duplicate nodes
    tx.run("MERGE (:Ingredient {name: $name})", name=ingredient_name)

# Assuming unique_ingredients is the list of unique ingredient names extracted from the dfs DataFrame
with driver.session() as session:
    for ingredient_name in unique_ingredients:  # Iterate over unique ingredients
        session.write_transaction(add_ingredient, ingredient_name)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3323882052.py:7: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3323882052.py:9: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_ingredient, ingredient_name)


In [40]:
# Step 2: Process Manufacturing Steps
# Create nodes for each unique manufacturing step, with the name and description properties 

def add_manufacturing_step(tx, step_name):
    # Use MERGE to create a unique node for the manufacturing step with only a name
    tx.run("""
        MERGE (:ManufacturingStep {name: $step_name})
    """, step_name=step_name)

# Extract unique manufacturing steps from the dfs DataFrame
manufacturing_steps = set()
for i in range(1, 6):  # Assuming there are 5 manufacturing steps columns
    manufacturing_steps.update(dfs[f'Manufacturing Step {i}'].dropna().unique())

# Create manufacturing step nodes in the Neo4j database
with driver.session() as session:
    for step_name in manufacturing_steps:
        if step_name:  # Check if step_name is not empty or null
            session.write_transaction(add_manufacturing_step, step_name)

print("Manufacturing step nodes have been created in the Neo4j database.")

C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3289840250.py:16: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3289840250.py:19: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_manufacturing_step, step_name)


Manufacturing step nodes have been created in the Neo4j database.


In [41]:
# Step 3: Create Relationships Between Ingredients and Manufacturing Steps
# - For each 'dfs' entry, link new ingredient nodes to corresponding manufacturing steps.
# - After the final step, connect to the Product node, matching the 'Product' column name with existing node names.
# attempted a long script, doing everything, which failed horribly - so much for trying to save time. 
# its created errors, so the current db looks bad. However, if we dump it and restart this script then all will be fine. 



In [42]:
# Iterate over the dfs DataFrame.
# For each ingredient, find its first manufacturing step.
# Create a "First_Step" relationship from the ingredient node to the first manufacturing step node.

def create_first_step_relationships(tx, ingredient_name, first_step):
    if not first_step or pd.isna(first_step):
        return
    
    tx.run("""
        MATCH (ing:Ingredient {name: $ingredient_name})
        MATCH (step:ManufacturingStep {name: $first_step})
        MERGE (ing)-[:First_Step]->(step)
    """, ingredient_name=ingredient_name, first_step=first_step)

with driver.session() as session:
    for index, row in dfs.iterrows():
        first_step = row['Manufacturing Step 1']  # Assuming this is the first manufacturing step
        # Iterate over ingredient columns
        for i in range(1, 7):  # Adjust the range as per your columns
            ingredient_name = row.get(f'Ingredient - {i}')
            if pd.notna(ingredient_name):
                session.write_transaction(create_first_step_relationships, ingredient_name, first_step)



C:\Users\apple\AppData\Local\Temp\ipykernel_10124\837884510.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\837884510.py:22: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_first_step_relationships, ingredient_name, first_step)


This step involves populating the Neo4j graph database with sequential relationships between manufacturing steps for each product. These relationships are annotated with properties that provide context about the manufacturing process.
##### Process
1. **Data Iteration**:
    
    - The script iterates over each row in the `dfs` DataFrame.
2. **Data Collection**:
    
    - Extracts a list of ingredients from `Ingredient - X` columns.
    - Gathers manufacturing steps and their descriptions from `Manufacturing Step X` and `Manufacturing Step X Description` columns.
3. **Data Cleaning**:
    
    - Filters out any `NaN` values to maintain data integrity.
4. **Neo4j Session**:
    
    - Initiates a session to communicate with the Neo4j database.
5. **Relationship Creation**:
    
    - Constructs relationships between consecutive manufacturing steps.
    - Assigns a sequence label (`STEP_2`, `STEP_3`, etc.) to each relationship.
    - Incorporates relationship properties:
        - `product_identifier`: Ties the step to a specific product.
        - `sequence_order`: Indicates the order of the step in the sequence.
        - `description`: Describes the action taken at each step.
        - `ingredients`: Lists the ingredients involved in the step.
        - Metadata: Includes `created_by`, `created_at`, `source`, `equipment`, and `revision`.

In [43]:
def create_sequential_step_relationships(tx, steps_info):
    for i in range(len(steps_info) - 1):
        current_step = steps_info[i]
        next_step = steps_info[i + 1]
        
        # Skip if any step is not defined
        if pd.isna(current_step) or pd.isna(next_step):
            continue

        # Define relationship type based on sequence order
        relationship_type = f"STEP_{i + 2}"

        # Execute Cypher query to create relationship
        tx.run(f"""
            MATCH (current:ManufacturingStep {{name: $current_step}})
            MATCH (next:ManufacturingStep {{name: $next_step}})
            MERGE (current)-[r:{relationship_type}]->(next)
        """, current_step=current_step, next_step=next_step)

# Prepare data and execute the function for each product
with driver.session() as session:
    for index, row in dfs.iterrows():
        # Collect all manufacturing steps for the current product row
        steps = [row[f'Manufacturing Step {i}'] for i in range(1, 6)]
        steps = [step for step in steps if not pd.isna(step)]  # Filter out any None or NaN values

        # Create sequential relationships
        session.write_transaction(create_sequential_step_relationships, steps)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\1365357593.py:21: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\1365357593.py:28: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_sequential_step_relationships, steps)


In [44]:
def add_product_and_ingredient_info(tx, product_name, ingredients, steps_info):
    for i in range(len(steps_info) - 1):
        current_step = steps_info[i]
        next_step = steps_info[i + 1]
        
        # Skip if any step is not defined
        if pd.isna(current_step) or pd.isna(next_step):
            continue

        # Define relationship type based on sequence order
        relationship_type = f"STEP_{i + 2}"

        # Execute Cypher query to add properties to existing relationship
        tx.run(f"""
            MATCH (current:ManufacturingStep {{name: $current_step}})-[r:{relationship_type}]->(next:ManufacturingStep {{name: $next_step}})
            SET r.product = $product_name,
                r.ingredients = $ingredients
        """, current_step=current_step,
            next_step=next_step,
            product_name=product_name,
            ingredients=ingredients
        )

# Prepare data and execute the function for each product
with driver.session() as session:
    for index, row in dfs.iterrows():
        product_name = row['Product']
        ingredients = [row[f'Ingredient - {i}'] for i in range(1, 7) if not pd.isna(row[f'Ingredient - {i}'])]
        steps = [row[f'Manufacturing Step {i}'] for i in range(1, 6)]
        steps = [step for step in steps if not pd.isna(step)]  # Filter out NaN values

        # Add product and ingredient information to relationships
        session.write_transaction(add_product_and_ingredient_info, product_name, ingredients, steps)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\4067986402.py:25: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\4067986402.py:33: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_product_and_ingredient_info, product_name, ingredients, steps)


In [45]:
# Link the Final Manufacturing Step to the Ingredient Product
# For each ingredient, identify the final manufacturing step, this can be variable.
# Connect the final manufacturing step to the ingredient node that represents the finished product.

def create_to_make_relationship(tx, ingredient_name, final_step):
    if not final_step or pd.isna(final_step):
        return

    # Create the "To_Make" relationship
    tx.run("""
        MATCH (step:ManufacturingStep {name: $final_step}), (ing:Ingredient {name: $ingredient_name})
        MERGE (step)-[:To_Make]->(ing)
    """, final_step=final_step, ingredient_name=ingredient_name)

# Execute the transaction for each row in the dataframe
with driver.session() as session:
    for index, row in dfs.iterrows():
        # Extract the ingredient name
        ingredient_name = row['Product']

        # Identify the final manufacturing step
        final_step = None
        for i in range(5, 0, -1):  # Reverse loop from step 5 to 1
            step = row[f'Manufacturing Step {i}']
            if pd.notna(step):
                final_step = step
                break

        # Create the "To_Make" relationship in Neo4j
        if final_step:
            session.write_transaction(create_to_make_relationship, ingredient_name, final_step)



C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3363913836.py:16: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3363913836.py:31: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_to_make_relationship, ingredient_name, final_step)


Create list of unique ingredients from dft
Create primary ingredients as a seperate node
if not a primary ingredient then merge to an ingredient node
create manufacturing steps as manufacturing nodes
create relationship from ingredient to manufacturing step
create relationship between the manufacturing step
the relationships have as properties the product, ingredient and the description of the manufacturing step.
the last manufacturing step is linked to the product (which is an ingredient node)


##### Step 1: Extract Unique Ingredients
Process 'dft' to create a list of unique ingredients from 'Ingredients - 1' to 'Ingredients - 5'.
Ensure there are no duplicates in comparison to existing ingredient nodes.

##### Step 2: Create or Merge Ingredient Nodes
For each ingredient, check the 'Primary Source' column.
If 'Primary Source' is 'Yes', create a new 'PrimaryIngredient' node.
If 'Primary Source' is 'No', merge with an existing 'Ingredient' node to avoid duplicates.
Utilize the MERGE statement in Cypher to handle both creation and merging efficiently.

##### Step 3: Link Ingredients to Manufacturing Steps
For each row in 'dft', identify the sequence of manufacturing steps and descriptions.
Create relationships between ingredient nodes and their corresponding manufacturing steps.
The relationship should be named according to the step sequence (e.g., 'STEP_1').

###### Properties for each relationship should include:
- product_identifier: Corresponds to the name from 'Product' column.
- sequence_order: Indicates the order of the manufacturing step.
- description: Descriptive text from the respective 'Description Step #' column.
- ingredients: A list of ingredients involved in each step, extracted from 'Ingredients - 1' to 'Ingredients - 5'.
- Additional properties such as timestamp, quantity, parameters, variations, and metadata can be added as placeholders for future use.
##### Step 4: Connect Manufacturing Steps
Link each manufacturing step to the subsequent step to create a chain.
The last step in the chain should be connected to the final product node with a 'TO_MAKE' relationship.

In [46]:
# Step 1: Extract unique ingredients
# Combine all ingredient columns into one series and drop NaN values
all_ingredients_series = pd.concat([dft[f'Ingredients - {i}'] for i in range(1, 6)]).dropna()

# Get unique ingredients
unique_ingredients = all_ingredients_series.unique().tolist()

# Split into primary and non-primary ingredients
primary_ingredients = dft[dft['Primary Source'] == 'Yes'][['Ingredients - 1', 'Ingredients - 2', 'Ingredients - 3', 'Ingredients - 4', 'Ingredients - 5']].melt()['value'].dropna().unique().tolist()
non_primary_ingredients = list(set(unique_ingredients) - set(primary_ingredients))

# Now there are two lists: 
# primary_ingredients contains all unique primary source ingredients
# non_primary_ingredients contains all unique non-primary source ingredients

In [47]:
# Step 2: Create Primary Ingredient node & merge Ingredient node.

def create_primary_ingredient_nodes(tx, primary_ingredients):
    for ingredient in primary_ingredients:
        tx.run("CREATE (:PrimaryIngredient {name: $ingredient_name})", ingredient_name=ingredient)

def merge_non_primary_ingredient_nodes(tx, non_primary_ingredients):
    for ingredient in non_primary_ingredients:
        tx.run("MERGE (:Ingredient {name: $ingredient_name})", ingredient_name=ingredient)

# Assuming you have two lists: primary_ingredients and non_primary_ingredients
with driver.session() as session:
    # Create nodes for primary ingredients
    session.write_transaction(create_primary_ingredient_nodes, primary_ingredients)
    
    # Merge nodes for non-primary ingredients
    session.write_transaction(merge_non_primary_ingredient_nodes, non_primary_ingredients)

C:\Users\apple\AppData\Local\Temp\ipykernel_10124\4009299719.py:12: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\4009299719.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_primary_ingredient_nodes, primary_ingredients)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\4009299719.py:17: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(merge_non_primary_ingredient_nodes, non_primary_ingredients)


In [48]:
# Step 3:

def create_ingredient_to_step_relationships(tx, dft):
    for index, row in dft.iterrows():
        product_name = row['Product']
        ingredients = [row[f'Ingredients - {i}'] for i in range(1, 6) if not pd.isna(row[f'Ingredients - {i}'])]
        first_step = row['Manufacturing Step 1']
        first_step_description = row['Description Step 1']

        for ingredient in ingredients:
            # Determine the label for the ingredient node based on 'Primary Source'
            label = "PrimaryIngredient" if row['Primary Source'] == 'Yes' else "Ingredient"

            tx.run(f"""
                MATCH (ing:{label} {{name: $ingredient}})
                MATCH (step:ManufacturingStep {{name: $first_step}})
                MERGE (ing)-[r:PROCESSED_IN]->(step)
                SET r.product = $product_name,
                    r.step_description = $first_step_description
            """, ingredient=ingredient, first_step=first_step, product_name=product_name, first_step_description=first_step_description)

# Execute the transaction
with driver.session() as session:
    session.write_transaction(create_ingredient_to_step_relationships, dft)



C:\Users\apple\AppData\Local\Temp\ipykernel_10124\23632003.py:23: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\23632003.py:24: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_ingredient_to_step_relationships, dft)


Extract Manufacturing Steps and Product Information:
- Loop through each row in the 'dft' DataFrame.
- For each product, identify the sequence of manufacturing steps and extract relevant information (names and descriptions).

Create Sequential Relationships Between Manufacturing Steps:
- For each product, link its manufacturing steps in sequence (e.g., STEP_1 to STEP_2, STEP_2 to STEP_3, and so on).
- Include properties in these relationships, such as the step number and description.
- Link Final Manufacturing Step to Product:

Determine the final manufacturing step for each product.
- Create a TO_MAKE relationship from this final step to the corresponding product node.
- This relationship can carry properties indicating the final step and its description.

In [49]:
# Create sequential relationships

def create_correct_step_relationships(tx, dft):
    for index, row in dft.iterrows():
        product_name = row['Product']
        ingredients = [row[f'Ingredients - {i}'] for i in range(1, 6) if not pd.isna(row[f'Ingredients - {i}'])]
        steps = [(row[f'Manufacturing Step {i}'], row[f'Description Step {i}']) for i in range(1, 6) if not pd.isna(row[f'Manufacturing Step {i}'])]

        # Create relationships between sequential steps
        for i in range(len(steps) - 1):
            current_step, current_description = steps[i]
            next_step, next_description = steps[i + 1]
            relationship_type = f"STEP_{i+1}"

            tx.run(f"""
                MATCH (current:ManufacturingStep {{name: $current_step}}), (next:ManufacturingStep {{name: $next_step}})
                MERGE (current)-[r:{relationship_type}]->(next)
                SET r.product_identifier = $product_name,
                    r.sequence_order = $i + 1,
                    r.description = $current_description,
                    r.ingredients = $ingredients
            """, current_step=current_step, next_step=next_step, product_name=product_name, i=i, current_description=current_description, ingredients=ingredients)

# Execute the transaction
with driver.session() as session:
    session.write_transaction(create_correct_step_relationships, dft)




C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2656296328.py:25: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\2656296328.py:26: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_correct_step_relationships, dft)


In [50]:
# link last manufacturing step to the product(dft) or ingredient node.

def link_final_step_to_product_as_ingredient(tx, dft):
    for index, row in dft.iterrows():
        product_name = row['Product']
        ingredients = [row[f'Ingredients - {i}'] for i in range(1, 6) if not pd.isna(row[f'Ingredients - {i}'])]
        final_step = None
        final_description = None

        # Identify the last non-empty manufacturing step
        for i in reversed(range(1, 6)):
            if not pd.isna(row[f'Manufacturing Step {i}']):
                final_step = row[f'Manufacturing Step {i}']
                final_description = row[f'Description Step {i}']
                break

        if final_step:
            # Create 'TO_MAKE' relationship from the final step to the product (Ingredient node)
            tx.run("""
                MATCH (step:ManufacturingStep {name: $final_step}), (prod:Ingredient {name: $product_name})
                MERGE (step)-[r:TO_MAKE]->(prod)
                SET r.description = $final_description, r.ingredients = $ingredients
            """, final_step=final_step, product_name=product_name, final_description=final_description, ingredients=ingredients)

# Execute the transaction
with driver.session() as session:
    session.write_transaction(link_final_step_to_product_as_ingredient, dft)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\12548206.py:26: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\12548206.py:27: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(link_final_step_to_product_as_ingredient, dft)


## Adding manufacturing steps to products

#### Identifying Unique Manufacturing Steps: 
Extract unique manufacturing steps from the 'Manufacturing Step #' columns to ensure each step is represented once in the graph.

#### Linking Products to Manufacturing Steps: 
For each product, create relationships to the manufacturing steps. These relationships will follow the sequence indicated by the step numbers.

#### Adding Relationship Properties: 
Include properties from the 'Description Step #' columns in the relationships to maintain context.

#### Incorporating Ingredients Information: 
Use the data from the 'Ingredients' column to add properties to the relationships, thus linking the product with the raw ingredients involved in its production.

In [51]:

# Extract unique manufacturing steps
manufacturing_steps = set()
for i in range(1, 9):  # We have 8 manufacturing steps
    manufacturing_steps.update(dfe[f'Manufacturing Step {i}'].dropna().unique())

# Now we have a set of unique manufacturing steps


In [52]:
# Function to merge manufacturing step nodes
def merge_manufacturing_step(tx, step_name, description):
    tx.run("""
        MERGE (step:ManufacturingStep {name: $step_name})
        ON CREATE SET step.description = $description
    """, step_name=step_name, description=description)


##### Properties of the relationship
- description: A text description of the manufacturing step, providing details about the process at that step.
- ingredients: A list or a string of ingredients involved in the manufacturing step, providing context for what is used or produced at that step.
- Product: Name of the final product.

In [53]:
# Create sequential relationships between manufacturing steps with the above properties
def create_relationships(tx, product_name, ingredients, steps_info):
    for i, step in enumerate(steps_info[:-1]):
        next_step = steps_info[i + 1]
        rel_type = f"STEP_{i + 1}"
        tx.run(f"""
            MATCH (s1:ManufacturingStep {{name: $step_name}}), (s2:ManufacturingStep {{name: $next_step_name}})
            MERGE (s1)-[r:{rel_type}]->(s2)
            SET r.product_identifier = $product_name,
                r.sequence_order = $sequence_order,
                r.description = $description,
                r.ingredients = $ingredients
            """,
            step_name=step[0],
            next_step_name=next_step[0],
            product_name=product_name,
            sequence_order=i + 1,
            description=step[1],
            ingredients=ingredients
        )

# Execute the transaction for each row in the dataframe
def add_relationships_from_dataframe(df):
    with driver.session() as session:
        for index, row in df.iterrows():
            product_name = row['Name']
            ingredients = row['Ingredients'].split(', ')  # Assuming ingredients are comma-separated
            steps_info = [(row[f'Manufacturing Step {i}'], row[f'Description Step {i}']) for i in range(1, 9) if not pd.isna(row[f'Manufacturing Step {i}'])]
            session.write_transaction(create_relationships, product_name, ingredients, steps_info)

#  dataframe 'dfe'
add_relationships_from_dataframe(dfe)

C:\Users\apple\AppData\Local\Temp\ipykernel_10124\1109305801.py:24: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\1109305801.py:29: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationships, product_name, ingredients, steps_info)


In [54]:
def link_final_step_to_product(tx, product_name, steps_info):
    # Find the last non-null step
    last_step = None
    for step in reversed(steps_info):
        if pd.notna(step[0]):
            last_step = step
            break

    if last_step:
        # Create a relationship from the last manufacturing step to the product
        tx.run("""
            MATCH (step:ManufacturingStep {name: $step_name}), (prod:Product {name: $product_name})
            MERGE (step)-[:PRODUCES]->(prod)
            """, step_name=last_step[0], product_name=product_name)

# Execute the transaction for each product
with driver.session() as session:
    for index, row in dfe.iterrows():
        product_name = row['Name']
        steps_info = [(row[f'Manufacturing Step {i}'], row[f'Description Step {i}']) for i in range(1, 9) if not pd.isna(row[f'Manufacturing Step {i}'])]
        session.write_transaction(link_final_step_to_product, product_name, steps_info)


C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3255757888.py:17: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
C:\Users\apple\AppData\Local\Temp\ipykernel_10124\3255757888.py:21: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(link_final_step_to_product, product_name, steps_info)
